# HD Parent Key Exposure: Example (c++)

<br>
<img src="images/hd_parent_exposure.jpg" alt="drawing" style="" width="700px"/>



### Libbitcoin-System and secp256k1 setup

In [2]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 

// secp256k1 supporting files.
#include "supporting_scripts/secp256k1_initializer.hpp"

### Derive new parent key pair and child private key.

In [3]:
// Generate m/44'/0'/0'/0 parent key from entropy.
// ---------------------------------------------------------------------------
{
    bc::data_chunk my_entropy_128(16);
    bc::pseudo_random_fill(my_entropy_128);

    // m/44'/0'/0'/0
    bc::wallet::hd_private m(my_entropy_128, bc::wallet::hd_private::mainnet);
    auto m_44h_0h_0h_0 = m.derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(0 + bc::wallet::hd_first_hardened_key)
        .derive_private(0 + bc::wallet::hd_first_hardened_key)
        .derive_private(0);

    uint32_t child_index(1);

    std::cout << "Parent Private Key: " << std::endl 
              << bc::encode_base16(bc::to_chunk(m_44h_0h_0h_0.secret()))
              << std::endl << std::endl 
              << "Parent Public Key:" << std::endl 
              << bc::encode_base16(bc::to_chunk(m_44h_0h_0h_0.to_public().point()))
              << std::endl << std::endl 
              << "Parent Chaincode:" << std::endl 
              << bc::encode_base16(bc::to_chunk(m_44h_0h_0h_0.to_public().chain_code()))
              << std::endl << std::endl 
              << "Child Private Key:" << std::endl 
              << bc::encode_base16(bc::to_chunk(m_44h_0h_0h_0.derive_private(child_index).secret()))
              << std::endl;
}

Parent Private Key: 
4c6ad5efc056af4ee2ea529d78bf7502c1ed7dfcc5906af7b0be85893ec37ed8

Parent Public Key:
02512a0fb2c76871513ff5b4f479b6bd327b3b31d5755fcae1ec6c134bf06d2d7a

Parent Chaincode:
363c5cdeb93f5d4339b564db991795ad9dc56434872cbdec0463724214905fc0

Child Private Key:
a5277ef99d48f2bd1a4098449494cfb20440901deae74c64c7fd4578b9a5ab7c


### Derive "parent private key" from "parent pubkey + chaincode" and "child private key".

In [4]:
{

    // 1) Parent public key, chaincode and child private key are exposed.
    // ---------------------------------------------------------------------------
    
    auto M_44h_0h_0h_0_point = bc::base16_literal(
        "02512a0fb2c76871513ff5b4f479b6bd327b3b31d5755fcae1ec6c134bf06d2d7a");
    auto M_44h_0h_0h_0_chaincode = bc::base16_literal(
        "363c5cdeb93f5d4339b564db991795ad9dc56434872cbdec0463724214905fc0");
    auto m_44h_0h_0h_0_1_secret = bc::base16_literal(
        "a5277ef99d48f2bd1a4098449494cfb20440901deae74c64c7fd4578b9a5ab7c");
    uint32_t child_index(1);
    
    // 2) Derive L256 from hmac_sha512_hash(parent public key||index, chaincode).
    // ---------------------------------------------------------------------------
    auto data = bc::splice(M_44h_0h_0h_0_point, bc::to_big_endian(child_index));
    auto my_byte_array_parts = bc::split(bc::hmac_sha512_hash(data, M_44h_0h_0h_0_chaincode));
    auto left_256 = my_byte_array_parts.left;

    // 3) Parent Private(m_44h_0h_0h_0) = child private(m_44h_0h_0h_0_1) - L256.
    // ---------------------------------------------------------------------------
    
    const auto context = bc::verification.context();
    if (secp256k1_ec_privkey_negate(context,left_256.data()) != 1)
        return 1;
    bc::ec_add(left_256, m_44h_0h_0h_0_1_secret);

    // Print out computed parent private key.
    // ---------------------------------------------------------------------------
    std::cout << "Computed Private Parent Key: " << std::endl
              << bc::encode_base16(bc::to_chunk(left_256)) << std::endl;
}

Computed Private Parent Key: 
4c6ad5efc056af4ee2ea529d78bf7502c1ed7dfcc5906af7b0be85893ec37ed8
